# Graph 

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))

In [2]:
from enum import Enum
class Scenario(Enum):
    single_object_single_robot = 0
    double_object_single_robot = 1
    single_object_dual_robot = 2
    assembly_3_piece = 3

In [3]:
current_scenario = Scenario.single_object_single_robot

In [4]:
if current_scenario in [Scenario.single_object_dual_robot, Scenario.assembly_3_piece]:
    from pkg.ur10_dual import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE, get_geometry_items_dict
else:
    from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE, get_geometry_items_dict
from pkg.constraint_graph import *
import matplotlib.pyplot as plt
from pkg.plot_utils import *
from pkg.utils import *
# from threading import Thread, Lock
from multiprocessing import Process, Lock, Manager
PROC_MODE = True

# simulate transitions

# Graph 

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))

In [2]:
from enum import Enum
class Scenario(Enum):
    single_object_single_robot = 0
    double_object_single_robot = 1
    single_object_dual_robot = 2
    assembly_3_piece = 3

In [3]:
current_scenario = Scenario.single_object_single_robot

In [4]:
if current_scenario in [Scenario.single_object_dual_robot, Scenario.assembly_3_piece]:
    from pkg.ur10_dual import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE, get_geometry_items_dict
else:
    from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE, get_geometry_items_dict
from pkg.constraint_graph import *
import matplotlib.pyplot as plt
from pkg.plot_utils import *
from pkg.utils import *
# from threading import Thread, Lock
from multiprocessing import Process, Lock, Manager
PROC_MODE = True

In [84]:
graph = ConstraintGraph(urdf_path = URDF_PATH, joint_names = JOINT_NAMES, link_names = LINK_NAMES)
# graph.set_fixed_geometry_items(get_geometry_items_dict(graph.urdf_content, color=(0,1,0,0.5), display=False, collision=True))
graph.set_fixed_geometry_items({
        k:[] for k, v in get_geometry_items_dict(graph.urdf_content, color=(0,1,0,0.5), display=False, collision=False).items()})

# Single object

In [85]:
if current_scenario == Scenario.single_object_single_robot:
    collision = False
    graph.add_geometry_items("world", 
                              [
                               GeoMesh(uri="package://my_mesh/meshes/stl/AirPick_cup_ctd.stl", BLH=(0.01,0.01,0.01), scale=(1e-3,1e-3,1e-3), name="gripper1", link_name="tool0", urdf_content=graph.urdf_content, color=(0.1,0.1,0.1,1), collision=collision),
                               GeoBox((0.5,-0.2,0.050), (0.1,0.1,0.1), name="box1", link_name="world", urdf_content=graph.urdf_content, color=(0.3,0.3,0.8,1), collision=collision),
                               GeoBox((0,0,-0.005), (3,3,0.01), name="floor", link_name="world", urdf_content=graph.urdf_content, color=(0.6,0.6,0.6,1), collision=collision),
#                                GeoBox((0.7,0.0,0.2), (0.7,0.05,0.4), name="wall", link_name="world", urdf_content=graph.urdf_content, color=(0.4,0.3,0.0,1), collision=collision),
#                                GeoBox((0.4,0.4,0.15), (0.15,0.15,0.3), name="stepper", link_name="world", urdf_content=graph.urdf_content, color=(0.4,0.3,0.0,1), collision=collision),
#                                GeoBox((0.4,0.4,0.3), (0.1,0.1,1e-3), name="goal_disp", link_name="world", urdf_content=graph.urdf_content, color=(0.8,0.0,0.0,1),
#                                       collision=False)
        ])
    
    graph.register_binder(name='vac1', _type=VacuumTool, point=[0,0,5e-2], link_name="tool0", direction=[0,0,1])
    graph.register_binder(name='floor', _type=PlacePlane, direction=[0,0,1])
#     graph.register_binder(name='goal', _type=PlaceFrame, point=(0.4,0.4,0.3+5e-4), link_name="world", orientation=[0,0,0])

    graph.register_object('box1', _type=BoxAction, binding=("bottom_p","floor"))

    graph.update_handles()

# Set RVIZ

In [86]:
graph.set_simulation(nWSR=50, regularization_factor= 1e-1)

publication OK
published: [0, 0, 0, 0, 0, 0]
publication OK
publication OK
publication OK


# build

In [87]:
graph.build_graph()

In [88]:
if current_scenario == Scenario.single_object_single_robot:
    graph.search_graph_ordered(initial_state = \
                                  State((('box1','bottom_p','floor'),), 
                                        {'box1': [0.5,-0.2,0.05,0,0,0,1]}, ZERO_JOINT_POSE),
                                  goal_state = State((('box1','top_p','vac1'),), None, None), display=False, 
                                  terminate_on_first = True, tree_margin = 0,
                                  depth_margin = 2, joint_motion_num=50, 
                                  **dict(N=300, dt=0.01, vel_conv=1e-2, err_conv=1e-3))

init: 	40.0 ms/5 = 7.998 ms 
run: 	3023.0 ms/5 = 604.54 ms 
init_sim: 	252.0 ms/5 = 50.354 ms 
put_constraints: 	23.0 ms/5 = 4.579 ms 
monitor: 	5.0 ms/5 = 0.997 ms 
simulate: 	2726.0 ms/5 = 545.112 ms 
output: 	0.0 ms/5 = 0.027 ms 
post: 	3.0 ms/6 = 0.435 ms 
test: 	2668.0 ms/4 = 667.052 ms 
start set transition: 	1.0 ms/1 = 1.314 ms 
set_simulate fun: 	38.0 ms/1 = 38.073 ms 


success - Goal cost:1->0 / Init cost:0->1 / branching: 0->1 (0.04 s)
node: (('box1', 'bottom_p', 'floor'),)->(('box1', 'top_p', 'vac1'),)
